In [1]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from PIL import Image , ImageTk
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline


# إنشاء النافذة الرئيسية
root = tk.Tk()
root.title("Underwater Human detection")
screen_width = root.winfo_screenwidth() 
screen_height = root.winfo_screenheight() 
root.geometry(f"{screen_width}x{screen_height}" )
root.resizable(width=False, height=False)
root.configure(background="#abadb0")
root.iconbitmap("D:\\senior project\\GUI\\images\\iconn.ico")


labelT = tk.Label(root, text="  Autonomous Underwater Vehicle extract the bodies from underwater to the beach  ",
bg ='#4b8ae3' , fg = 'white' ,font=("Times New Roman", 30, "italic") )
labelT.place(x = 0 , y = 0)
labelT.pack()



#frame0
frame0 = tk.Frame(root, height=665 , width = 1335 )
frame0.pack(side="top", padx=10, pady = 10)



#frame 1
frame1 = tk.LabelFrame(root, text="Unlabeled Image", height=400 , width = 400 , bg = '#4b8ae3')
frame1.pack(side="left", expand=True, padx=20, pady=20)
frame1.place(x = 230 , y = 100)

#frame2
frame2 = tk.LabelFrame(root, text="labeled Image", height=400 , width = 410 , bg = '#4b8ae3')
frame2.pack(side="left", expand=True, padx=20, pady=20)
frame2.place(x = 770 , y = 100)


# functions 

#fun1
def open_file_dialog():
    global input_image_path
    input_image_path = filedialog.askopenfilename(initialdir = "/",title = "Select Image File",filetypes = (("JPEG files","*.jpg"),("PNG files","*.png"),("All Files","*.*")))
    input_image = cv2.imread(input_image_path)
    input_image = cv2.resize(input_image, (400, 400))
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    input_image = Image.fromarray(input_image)
    input_image = ImageTk.PhotoImage(image=input_image)
    img_label = tk.Label(frame1)
    img_label.configure(image=input_image)
    img_label.image = input_image
    img_label.pack()  
    
    
    return input_image_path

pathh = open_file_dialog()   

# button 1
select_image_btn = tk.Button(frame0, text=" Load Image ", bg = '#4b8ae3' , fg = 'white', font=("Times New Roman", 12, "bold"),
command = open_file_dialog )
select_image_btn.pack(pady=10)
select_image_btn.place(x = 520 , y = 500)    








#func 2

def tflite_detect_images(modelpath = 'D:\\senior project\\last inshaa allah\\detect.tflite' , imgpath = pathh,
 lblpath = 'D:\\senior project\\last inshaa allah\\label.txt' ,savepath = 'D:\\senior project\\applicable data\\test', min_conf=0.75):

  # Grab filenames of all images in test folder
  #images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  
  image = cv2.imread(pathh)
  imH, imW, _ = image.shape 
  image_resized = cv2.resize(image, (width, height))
  input_data = np.expand_dims(image_resized, axis=0)

      # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
  if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Perform the actual detection by running the model with the image as input
  interpreter.set_tensor(input_details[0]['index'],input_data)
  interpreter.invoke()

      # Retrieve detection results
  boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
  classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
  scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

  detections = []

      # Loop over all detections and draw detection box if confidence is above minimum threshold
  for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Get bounding box coordinates and draw box
              # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
              ymin = int(max(1,(boxes[i][0] * imH)))
              xmin = int(max(1,(boxes[i][1] * imW)))
              ymax = int(min(imH,(boxes[i][2] * imH)))
              xmax = int(min(imW,(boxes[i][3] * imW)))
              
              cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

              # Draw label
              object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
              label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
              label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
              cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

              detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])

      
      
  
  
  patho = 'D:\\senior project\\applicable data\\test\\image.jpg'
  cv2.imwrite(patho, image)

  return patho

lblpath = tflite_detect_images()



# button 2
select_image_btn2 = tk.Button(frame0, text="Apply Model", bg = '#4b8ae3' , fg= 'white' , font=("Times New Roman", 12, "bold"),
command = tflite_detect_images )
select_image_btn2.pack(pady=10)
select_image_btn2.place(x = 650 , y = 500)


def show_label(path = lblpath):
  
    input_image = cv2.imread(lblpath)
    input_image = cv2.resize(input_image, (400, 400))
    input_image = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
    input_image = Image.fromarray(input_image)
    input_image = ImageTk.PhotoImage(image=input_image)
    img_label2 = tk.Label(frame2)
    img_label2.configure(image=input_image)
    img_label2.image = input_image
    img_label2.pack()  
    return





# button 3 
select_image_btn3 = tk.Button(frame0, text=" Show Label " , bg = '#4b8ae3', fg = 'white' ,font=("Times New Roman", 12, "bold"),
 command = show_label )
select_image_btn3.pack(pady=10)
select_image_btn3.place(x = 780 , y = 500)



root.mainloop()

TclError: bitmap "D:\senior project\GUI\images\iconn.ico" not defined